# 导入氨基酸blosum62矩阵

In [1]:
import scipy.io as sio
blosum = sio.loadmat('e:/repoes/jci/bio/blosum.mat')
blosumMatrix = blosum['blosum62']
alphabet = 'ARNDCQEGHILKMFPSTWYVBZX*'

In [2]:
import numpy as np
def sigmoidfun(x):
    return 1/(1 + np.exp(-x))

In [3]:
# 氨基酸编码字典
amino_acid_blosum_dict = {}
for a in alphabet:
    amino_acid_blosum_dict[a] = sigmoidfun(blosumMatrix[alphabet.index(a),:])

# 导入包

In [4]:
import sys
sys.path.append('e:/Repoes/jci/')
sys.path.append('e:/Repoes/jci/bio/')
sys.path.append('..')

In [5]:
from Bio import SeqIO
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks

In [6]:
from tools import displayMetrics, displayMLMetrics, plot_history

# 定义模型函数

In [7]:
"""
位置函数
    基于角度的位置编码方法。计算位置编码矢量的长度
    Parameters
    ----------
    pos : 
        在句子中字的位置序号，取值范围是[0, max_sequence_len).
    i   : int
        字向量的维度，取值范围是[0, embedding_dim).
    embedding_dim : int
        字向量最大维度， 即embedding_dim的最大值.

    Returns
    -------
    float32
        第pos位置上对应矢量的长度.
"""
def get_angles(pos, i, embed_dim):
    angel_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(embed_dim))
    return pos * angel_rates

In [8]:
# 位置编码
def position_encoding(position, embed_dim):
    angel_rads = get_angles(np.arange(position)[:, np.newaxis], 
                            np.arange(embed_dim)[np.newaxis, :], 
                            embed_dim)
    sines = np.sin(angel_rads[:, 0::2])
    cones = np.cos(angel_rads[:, 1::2])
    pos_encoding = np.concatenate([sines, cones], axis=-1)
    #pos_encoding = pos_encoding[np.newaxis, ...]
    return pos_encoding

In [9]:
# 将padding位mark，原来为0的padding项的mark输出为1
def create_padding_mask(seq):
    # 获取为0的padding项
    seq = tf.cast(tf.math.equal(tf.reduce_sum(seq,axis=-1), 0), tf.float32)
    # 扩充维度以便用于attention矩阵
    return seq[:, np.newaxis, np.newaxis, :] # (batch_size, 1, 1, seq_len)


In [10]:
# 生成[batch_size, maxlen, embed_dim]的矩阵掩码
def code_and_padding(seqs, padding_position="post", maxlen=1000):
    #position = position_encoding(maxlen, 24)
    amino_acids = "ARNDCQEGHILKMFPSTWYVX"
    regexp = re.compile('[^ARNDCQEGHILKMFPSTWYVX]')
    X = []
    if padding_position == "post":
        for seq in seqs:
            seq = regexp.sub('X', seq)
            seq = "Z" + seq + "*"
            i = 0
            t = []
            while i < len(seq) and i < maxlen:
                t.append(amino_acid_blosum_dict[seq[i]])
                i += 1
            while i < maxlen:
                t.append(np.zeros((24,)))
                i += 1
            X.append(t)
    elif padding_position == "pre":
        for seq in seqs:
            seq = regexp.sub('X', seq)
            seq = "Z" + seq + "*"
            i = -1
            t = []
            while abs(i) <= len(seq) and abs(i) <= maxlen:
                t.insert(0, amino_acid_blosum_dict[seq[i]])
                i -= 1
            while abs(i) <= maxlen:
                t.insert(0, np.zeros((24,)))
                i -= 1
            X.append(t)
    return np.array(X)

In [11]:
# 自注意力机制
def scaled_dot_product_attention(q, k, v, mask):
    
    # query key 相乘获取匹配关系
    matmul_qk = tf.matmul(q, k, transpose_b=True)
    
    # 使用dk进行缩放
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
    
    # 掩码.将被掩码的token乘以-1e9（表示负无穷），这样
    # softmax之后就为0， 不对其它token产生影响
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)
    # 通过softmax获取attention权重
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
    # attention乘上value
    output = tf.matmul(attention_weights, v) # (..., seq_len_v, depth)
    
    return output, attention_weights

# 构造mutil head attention层

multi-head attention包含3部分：线性层与分头-缩放点积注意力-头链接-末尾线性层
每个多头注意块有三个输入：Q（查询），K（密钥），V（值）。它们通过第一层线性层并分成多个头
Q，K，V不是一个单独的注意头，而是分成多个头，因为它允许模型共同参与来自不同表征空间的
不同信息。在拆分后，每个头部具有降低的维度，总计算成本与具有全维度的单个头部注意力相同


In [12]:
class MultiHeadAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.embed_dim = embed_dim
        
        # embd_dim必须可以被num_heads整除
        assert embed_dim % num_heads == 0
        # 分头后的维度
        self.projection_dim = embed_dim // num_heads
        self.wq = layers.Dense(embed_dim)
        self.wk = layers.Dense(embed_dim)
        self.wv = layers.Dense(embed_dim)
        self.dense = layers.Dense(embed_dim)
        
    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]
        
        # 分头前的前向网络，获取q, k, v语义
        q = self.wq(q)
        k = self.wq(k)
        v = self.wv(v)
        
        # 分头
        q = self.separate_heads(q, batch_size) # [batch_size, num_heads, seq_len_q, projection_dim]
        k = self.separate_heads(k, batch_size)
        v = self.separate_heads(v, batch_size)
        
        # 通过缩放点积注意力层
        scaled_attention, attention_weights = scaled_dot_product_attention(q, k, v, mask)
        # 把多头维度后移
        scaled_attention = tf.transpose(scaled_attention, [0, 2, 1, 3])
        # 合并多头
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embed_dim))
        
        # 全连接重塑
        output = self.dense(concat_attention)
        
        return output, attention_weights

# 构造Transformer层

In [13]:
# 构造前向网络
def point_wise_feed_forward_network(d_model, diff):
    # d_model 即embed_dim
    return tf.keras.Sequential([
        layers.Dense(diff, activation='relu'),
        layers.Dense(d_model)])


In [14]:
# transformer编码层
class EncoderLayer(layers.Layer):
    def __init__(self, d_model, n_heads, ffd, dropout_rate=0.1):
        super(EncoderLayer, self).__init__()
        self.mha = MultiHeadAttention(d_model, n_heads)
        self.ffn = point_wise_feed_forward_network(d_model, ffd)
        
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        
        self.dropout1 = layers.Dropout(dropout_rate)
        self.dropout2 = layers.Dropout(dropout_rate)
        
    def call(self, inputs, training, mask):
        att_output, _ = self.mha(inputs, inputs, inputs, mask)
        att_output = self.dropout1(att_output, training=training)
        out1 = self.layernorm1(inputs + att_output)
        # 前向网络
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)
        
        return out2

# transformer编码块
一个transformer编码块包含n_layers个transformer层

In [15]:
class Encoder(layers.Layer):
    def __init__(self, n_layers, d_model, n_heads, ffd,
                 input_vocab_size, max_seq_len, dropout_rate=0.1):
        super(Encoder, self).__init__()
        
        self.n_layers = n_layers
        self.d_model = d_model
        self.pos_embedding = position_encoding(max_seq_len, d_model)
        self.encoder_layer = [EncoderLayer(d_model, n_heads, ffd, dropout_rate)
                              for _ in range(n_layers)]
        self.dropout = layers.Dropout(dropout_rate)
        
    def call(self, inputs, training, mask):
        word_emb = tf.cast(inputs, tf.float32)
        word_emb *= (tf.cast(self.d_model, tf.float32))
        emb = word_emb + self.pos_embedding
        x = self.dropout(emb, training=training)
        for i in range(self.n_layers):
            x = self.encoder_layer[i](x, training, mask)
        return x

# 建立预测模型
模型以transformer编码块为上游任务，从Transformer的编码块中提取特征，然后加上全连接层构建神经网络

In [16]:
def buildModel(maxlen, vocab_size, embed_dim, num_heads, ff_dim, 
               num_blocks, droprate, fl_size, num_classes):
    inputs = layers.Input(shape=(maxlen,24))
    
    encode_padding_mask = create_padding_mask(inputs)
    encoder = Encoder(n_layers=num_blocks, d_model=embed_dim, n_heads=num_heads, 
                      ffd=ff_dim, input_vocab_size=vocab_size, 
                      max_seq_len=maxlen, dropout_rate=droprate)
    x = encoder(inputs, False, encode_padding_mask)
    
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dropout(droprate)(x)
    x = layers.Dense(fl_size, activation="relu")(x)
    x = layers.Dropout(droprate)(x)
    
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile("adam", "categorical_crossentropy", metrics=["accuracy"])
    return model

In [17]:
def transformer_predictor(X_train, y_train, X_test, y_test, modelfile, params):
    keras.backend.clear_session()

    model = buildModel(params['maxlen'], params['vocab_size'], params['embed_dim'], 
                    params['num_heads'], params['ff_dim'],  params['num_blocks'], 
                    params['droprate'], params['fl_size'], params['num_classes'])
    model.summary()

    checkpoint = callbacks.ModelCheckpoint(modelfile, monitor='val_loss',
                                       save_best_only=True, 
                                       save_weights_only=True, 
                                       verbose=1)
    history = model.fit(
        X_train, y_train, 
        batch_size=params['batch_size'], epochs=params['epochs'], 
        validation_data=(X_test, y_test),
        callbacks=[checkpoint]
        )

    plot_history(history)

    #model.load_weights(modelfile)
    score = model.predict(X_test)
    
    return score

# 定义模型参数

In [18]:
# transformer net params
params = {}
params['vocab_size'] = 24
params['maxlen'] = 500
params['embed_dim'] = 24 # Embedding size for each token
params['num_heads'] = 4  # Number of attention heads
params['ff_dim'] = 128  # Hidden layer size in feed forward network inside transformer
params['num_blocks'] = 12
params['droprate'] = 0.2
params['fl_size'] = 128
params['num_classes'] = 2
params['epochs'] = 20
params['batch_size'] = 32

## 读入数据

In [19]:
def load_seqs():
    """
    read Enzyme and not Enzyme sequences, 
    in which every protein sequence is less than 40% similarity with others.

    Returns
    -------
    seqs:
        protein sequences
    labels:
        if 0 for not Enzyme, else 1 for Enzyme

    """
    
    # read Enzyme and not Enzyme sequences 
    seq_records = SeqIO.parse('data/EC_40.fasta', 'fasta')
    seq_records = shuffle(list(seq_records), random_state=42)
    Enzyme_seqs = []
    for seq_record in seq_records:
        if len(str(seq_record.seq)) >= 50:
            Enzyme_seqs.append(str(seq_record.seq))
            
    seq_records = SeqIO.parse('data/NotEC_40.fasta', 'fasta')
    seq_records = shuffle(list(seq_records), random_state=42)
    notEnzyme_seqs = []
    for seq_record in seq_records:
        if len(str(seq_record.seq)) >= 50:
            notEnzyme_seqs.append(str(seq_record.seq))
    notEnzyme_seqs = shuffle(notEnzyme_seqs)
    notEnzyme_seqs = notEnzyme_seqs[:len(Enzyme_seqs)]
    
    
    seqs = Enzyme_seqs + notEnzyme_seqs
    labels = [1 for i in range(len(Enzyme_seqs))] + [0 for i in range(len(notEnzyme_seqs))]

    return seqs, labels

In [20]:
# load data
seqs, labels = load_seqs()
# split data into train and test
seqs_train, seqs_test, labels_train, labels_test = train_test_split(seqs, labels, 
                                                    test_size=0.3, 
                                                    random_state=42,
                                                    stratify=labels)

In [21]:
x_train = code_and_padding(seqs_train, maxlen=params['maxlen'])
x_test = code_and_padding(seqs_test, maxlen=params['maxlen'])

In [22]:
y_train = keras.utils.to_categorical(labels_train, params['num_classes'])
y_test = keras.utils.to_categorical(labels_test, params['num_classes'])

## 训练和测试

In [23]:
# training and test
modelfile = './model/ec/ec_trainsformer_{}_{}.h5'.format(params["maxlen"], "pos")
score = transformer_predictor(x_train, y_train, x_test, y_test, modelfile, params)
pred = np.argmax(score, 1)
displayMetrics(np.argmax(y_test, 1), pred)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 500, 24)]         0         
_________________________________________________________________
tf_op_layer_Sum (TensorFlowO [(None, 500)]             0         
_________________________________________________________________
tf_op_layer_Equal (TensorFlo [(None, 500)]             0         
_________________________________________________________________
tf_op_layer_Cast (TensorFlow [(None, 500)]             0         
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 1, 1, 500)]       0         
_________________________________________________________________
encoder (Encoder)            (None, 500, 24)           98304     
_________________________________________________________________
global_max_pooling1d (Global (None, 24)                0     

KeyboardInterrupt: 